In [1]:
import pandas as pd
import numpy as np
from yahoo_fin.stock_info import get_data
import datetime

### Defined the directory and model parameter

In [2]:
log_directory = "strategy_log/"
file_directory = "data/"

In [3]:
look_back = 3
portfolio_size = 10

### Extract the data from Yahoo Finance

In [4]:
with open(file_directory+"yahoo_finance_crypto_list.txt") as f:
    
    ticker_list = f.readlines()
    
    ticker_list = [ticker.strip() for ticker in ticker_list]

In [5]:
start_date = "2021-01-05"
end_date = "2021-01-16"

In [ ]:
start_date = (datetime.datetime.today()-datetime.timedelta(days=look_back)).strftime("%Y-%m-%d")
end_date = (datetime.datetime.today()).strftime("%Y-%m-%d")

In [6]:
price_dic = {}
error_list = []

for ticker in ticker_list:
    
    try:
        price_df = get_data(ticker=ticker,
                        start_date=start_date,
                        end_date=end_date,
                        index_as_date=True,
                        interval="1d")
        
        price_dic[ticker] = price_df
    
    except:
        
        print("Error.")
        error_list.append(ticker)
    
    #time.sleep(1)

Error.
Error.
Error.
Error.
Error.


In [7]:
price_df_list = [price_dic[ticker] for ticker in ticker_list if ticker not in error_list]
price_df = pd.concat(price_df_list,axis=0)

### Filter the universe by average transaction volume

In [8]:
average_volume = price_df[["volume","ticker"]].groupby("ticker").mean()
volume_threshold = average_volume.describe().loc["25%",:][0]
universe_list = list(average_volume[average_volume>volume_threshold].index)

In [9]:
price_df_list = [price_dic[ticker] for ticker in universe_list]
price_df = pd.concat(price_df_list,axis=0)

### Calculate Signal

In [10]:
price_df["date"] = price_df.index
price_wide = price_df.pivot_table(values="close",columns="ticker",index="date")


signal = price_wide.pct_change(look_back)
signal = signal.iloc[-1,:].dropna()

In [11]:
signal

ticker
ABBC-USD    0.151906
ACH-USD     0.003312
ACT-USD     0.058953
ADA-USD     0.123833
ADK-USD     0.008405
              ...   
ZEN-USD    -0.093492
ZIL-USD     0.045322
ZNN-USD     0.268891
ZRX-USD     0.042972
ZYN-USD    -0.019635
Name: 2021-01-16 00:00:00, Length: 366, dtype: float64

### Create Portfolios

In [12]:
top_portfolio = signal.nlargest(portfolio_size)
bottom_portfolio = signal.nsmallest(portfolio_size)

top_portfolio = pd.DataFrame(top_portfolio).T
bottom_portfolio = pd.DataFrame(bottom_portfolio).T

In [13]:
bottom_portfolio

ticker,MTC1-USD,MTC2-USD,JUL-USD,DUN-USD,BPS-USD,GCC1-USD,GCC2-USD,SCC3-USD,SCC4-USD,NVT-USD
2021-01-16,-0.550414,-0.550414,-0.516913,-0.470114,-0.394037,-0.352615,-0.352615,-0.347915,-0.347915,-0.310488


In [14]:
top_portfolio

ticker,LYNX-USD,AIB-USD,MBC-USD,XRC-USD,CRU-USD,PHA-USD,VIN-USD,VGX-USD,DOT1-USD,DOT2-USD
2021-01-16,5.281575,4.051276,1.73065,1.445965,1.230299,1.080479,0.712248,0.68131,0.659144,0.659144


### Write the Log

In [15]:
top_portfolio.to_csv(log_directory+"top_portfolio"+end_date+".csv")
bottom_portfolio.to_csv(log_directory+"bottom_portfolio"+end_date+".csv")

In [16]:
end_date

'2021-01-16'